### Planning Agent / Send Sucky Home Efficiently

So far you have

1.  Your GoHomeAgent from Lab 1.  It explores around and sucks dirt until its battery reaches a threshold level, then tries to find its way back to (1,1).  Lacking the ability to plan, it does not do so very well
2.  The PlanningAgent introduced the concept of a *path*  -- a list of squares from A to B such that the path begins at A and ends at B and every square in the list is not a wall, and is *adjacent* to the ones surrounding it.  The planning agent also has the code to make Sucky follow a path.
3.  The module ``shortestpath.py`` builds a path, and it is missing.  It implements a function ```shortest_path(source_pos, dest_pos, free_squares)```.  You need to implement that function using the search module from Lab 2.   This notebook will help you do write that function.

Once you have the ```shortest_path``` function working, you will then copy it to a file ```shortestpath.py``` and you will copy in your ```GoHomeAgent``` from lab 1, and you will modify the agent so when its battery reaches threshold it will plan a path from where it is to (1,1), then execute the plan, then execute the action ```ACTION_STOP```



In [1]:
from agents.searchClientInterface import WorldState
from agents.searchClientInterface import Problem
from agents.searchClientInterface import BFSEvaluator
from agents.searchFramework import aStarSearch
import copy

In [13]:
##  Build your world state.
##  HINT.  Your "plan" is a sequence of square coordinates.  Your "current state"
##  is the state the planning agent is in.  

class ShortestPathWorldState(WorldState):
    
    def __init__(self, start_square, free_squares):
        self._current_square = start_square
        self._free_squares = set(free_squares)
    
    # Convenience function to make these objects print nicely
    def __str__(self):
        return "{" + str(self._square) + "}"
    
    def __eq__(self, other):
        return self._current_square == other._current_square

    def __hash__(self):
        return hash(self._current_square)
    
    #  Create a successor state for every square that is adjacent 
    #  to the current state's current position
    
    def successors(self):
        return [self.make_adjacent_state(adj) for adj in self.adjacencies(self._current_square)] 
    
    #  This is the state where we move from _square to the square
    #  Remember this function has to return a pair (newState, action)
    #  For our path planning, the "action" will be the next square in the path.
    #  For example, if the current square is (2,2) and the adjacency passed in is (1,2)
    #  then this function would return (<state where current square is (1,2)>, (1,2))
    #  That is, in the new state the agent is at square (1,2) and (1,2) is also added
    #  to the action list
    
    def make_adjacent_state(self, adjacency):
        # Check if the adjacency is a valid move
        if adjacency in self._free_squares:
            new_state = ShortestPathWorldState(adjacency, self._free_squares - {adjacency})
            return new_state, adjacency
        else:
            return None
    
    # Return a list of all the squares in the free_squares list that are adjacent
    # to the square passed as parameter.
    # For example if the "current square" is (2,2) then all the adjacent squares
    # are (1,2), (2,1), (2,3), (3,2) -- but you want to return only those squares
    # that are on the free_square_list that was passed in via the constructor

    def adjacencies(self, square):
        adjacent_squares = [(square[0] - 1, square[1]), (square[0] + 1, square[1]),
                            (square[0], square[1] - 1), (square[0], square[1] + 1)]
        return [adj for adj in adjacent_squares if adj in self._free_squares]


In [14]:
#  Now define the problem:  a problem is the initial square, the destination square, 
#  and the list of free squares

class ShortestPathProblem(Problem):
    
    def __init__(self, initial_square, dest_square, free_squares):
        self.initial_square = initial_square
        self.dest_square = dest_square
        self.free_squares = set(free_squares) 

    def initial(self):
        return ShortestPathWorldState(self.initial_square, self.free_squares)

    def isGoal(self, state):
        return state._current_square == self.dest_square

In [15]:
##  Your shortest path function is just a call to aStarSearch
##  Remember, shortest_path should return (just) the path:  a list 
##  of squares starting at the source and ending at the destination, 
##  and passing through adjacent squares in the free_square_list

def shortest_path(source_square, dest_square, free_square_list):
    # Check if the source and destination are the same
    if source_square == dest_square:
        return []

    # Create a ShortestPathProblem instance with the given parameters
    problem = ShortestPathProblem(source_square, dest_square, free_square_list)

    # Perform A* search using BFSEvaluator
    solution = aStarSearch(problem, BFSEvaluator())

    # Extract and return the path from the solution
    path = solution[0] if solution else []  # If solution is None, return an empty path
    return path


Example for an agent who is at (4,4) and wants to go home

```
source_square = (4,4)
dest_square = (1,1)
free_squares = [(1,1), (2,1), (2,2), (1,3), (1,4), (2,3), (2,4), (3,4), (4,4)]
print(shortest_path(source_square, dest_square, free_squares))

[(3, 4), (2, 4), (2, 3), (2, 2), (2, 1), (1, 1)]
```

In [16]:
## Test your code on the example above

source_square = (4,4)
dest_square = (1,1)
free_squares = [(1,1), (2,1), (2,2), (1,3), (1,4), (2,3), (2,4), (3,4), (4,4)]

# Test the shortest_path function
result_path = shortest_path(source_square, dest_square, free_squares)

# Print the result
print(result_path)

[(3, 4), (2, 4), (2, 3), (2, 2), (2, 1), (1, 1)]
